In [6]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext,SparkConf
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql.functions import *
import os
import boto3

In [2]:
sys.argv +=["--JOB_NAME","glue-script","--landing_bucket","gbs-sap-p2p-logistics-landing-dev","--SECURITY_CONFIGURATION","P2pLogisticsGlueSecurityConfiguration","--stagingbucket","gbs-p2p-logistics-staging-dev","--stagingfolder","testingfromlocal"]
args = getResolvedOptions(sys.argv, ['JOB_NAME','landing_bucket','stagingbucket','stagingfolder'])
print(args)

{'job_bookmark_option': 'job-bookmark-disable', 'job_bookmark_from': None, 'job_bookmark_to': None, 'JOB_ID': None, 'JOB_RUN_ID': None, 'SECURITY_CONFIGURATION': 'P2pLogisticsGlueSecurityConfiguration', 'encryption_type': None, 'RedshiftTempDir': None, 'TempDir': None, 'JOB_NAME': 'glue-script', 'landing_bucket': 'gbs-sap-p2p-logistics-landing-dev', 'stagingbucket': 'gbs-p2p-logistics-staging-dev', 'stagingfolder': 'testingfromlocal'}


In [3]:
sc = SparkContext()
glueContext = GlueContext(sc)
glueContext._sc._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
glueContext._sc._jsc.hadoopConfiguration().set("fs.s3a.access.key",os.environ['AWS_ACCESS_KEY_ID'])
glueContext._sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key",os.environ['AWS_SECRET_ACCESS_KEY'] )
glueContext._sc._jsc.hadoopConfiguration().set("fs.s3a.session.token",os.environ['AWS_SESSION_TOKEN'])
glueContext.spark_session.conf.set("spark.local.dir", "C:\\rpa-logs\\")
job = Job(glueContext)
job.init(args['JOB_NAME'], args)
spark = glueContext.spark_session
datasource0 = glueContext.create_dynamic_frame_from_options(connection_type="s3",
                                                           format="csv",
                                                           connection_options = {"paths": [f"s3://{args['landing_bucket']}/APPROVALS/APPROVALS_2020-07-02 113825.csv.gz"]},
                                                           format_options = {"withHeader": True, "separator": ";","escaper": '"', "quoteChar": '"'},
                                                           transformation_ctx = "datasource0")



In [5]:
datasource0.show()

In [13]:
s3=boto3.client("s3")
s3.list_buckets()

{'ResponseMetadata': {'RequestId': '64E225FC4B7AC687',
  'HostId': 'Q074pZ9Iba1LJ88SXKTlJeN8WNjg0lw5VWiephPyGtk6tS2I/rR/GhpQC1cNixhPxUK+P0Ih6+8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Q074pZ9Iba1LJ88SXKTlJeN8WNjg0lw5VWiephPyGtk6tS2I/rR/GhpQC1cNixhPxUK+P0Ih6+8=',
   'x-amz-request-id': '64E225FC4B7AC687',
   'date': 'Mon, 20 Jul 2020 07:36:25 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aws-athena-query-results-data-analytics-dev',
   'CreationDate': datetime.datetime(2020, 6, 30, 11, 54, 32, tzinfo=tzlocal())},
  {'Name': 'd2go-gbs-o-ds-aa-dev-data-bucket',
   'CreationDate': datetime.datetime(2020, 6, 25, 23, 42, 11, tzinfo=tzlocal())},
  {'Name': 'd2go-gbs-o-ds-aa-dev-tfstate',
   'CreationDate': datetime.datetime(2020, 6, 25, 23, 32, 52, tzinfo=tzlocal())},
  {'Name': 'data-analytics-cloudformation-dev',
   'CreationDate': datetime.datetime(2020, 6, 30, 11, 

In [11]:
s3